##  Demonstration: Tool-Calling Assistant using Model Context Protocol (MCP)

This segment of the notebook demonstrates how to use the **Model Context Protocol (MCP)** format with OpenAI `gpt-4-0613` model to simulate a tool-using assistant.

###  Objective
The assistant is asked a simple math question:  
**“What is the sum of 12 and 30?”**  
The assistant is allowed to call a custom-defined tool (`get_sum`) to compute the answer.

---

###  Code Breakdown

#### **Step 1: Load API key**
Loads the OpenAI API key from a `.env` file using `dotenv`.

#### **Step 2: Define the Tool**
A tool named `get_sum` is declared. It:
- Takes two numbers `a` and `b`
- Returns their sum

This tool is made available to the model via the `tools` argument.

#### **Step 3: Create the Message History**
The conversation starts with a `system` prompt (setting assistant behavior), followed by a `user` question:  
_"What's the sum of 12 and 30?"_

#### **Step 4: First Model Call**
The assistant receives the user input and decides whether to:
- Answer directly, **or**
- Use a tool (in this case, `get_sum`)  

The tool choice is left to the model using `tool_choice="auto"`.

#### **Step 5: Process the Model’s Decision**
- If a tool call is returned, it’s printed (function name, arguments, and ID).
- If not, the assistant's direct response is printed.

#### **Step 6: Simulate the Tool Execution**
We simulate the actual tool execution by:
- Parsing the arguments with `json.loads`
- Computing the result (`a + b`)
- Packaging the result as a response to the assistant

#### **Step 7: Extend the Message History**
The tool call and its result are appended to the conversation. This mirrors what would happen if a backend system executed the tool and returned the output to the LLM.

#### **Step 8: Second Model Call**
The assistant sees the tool’s result and responds to the user. This step closes the loop with a natural-language answer.

#### **Step 9: Print Final Answer**
Displays the final assistant response, demonstrating how the LLM:
- Called a tool
- Interpreted the result
- Answered the original question

---

###  Summary of MCP Interaction

1. Assistant receives the user query
2. Assistant decides to call the `get_sum` tool
3. Tool returns: `{ "result": 42 }`
4. Assistant replies:  
   **“The sum of 12 and 30 is 42.”**

This workflow showcases **multi-turn structured interactions** via **MCP**, allowing for **tool use**, **state tracking**, and **transparent reasoning**.

---




In [26]:
import os
from dotenv import load_dotenv
from openai import OpenAI
import json

# === STEP 1: Load OpenAI API key from local .env file ===
load_dotenv("keys.env")
client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

# === STEP 2: Define available tool(s) ===
# The assistant can use this tool if it determines that it needs to perform a computation
tools = [{
    "type": "function",
    "function": {
        "name": "get_sum",
        "description": "Returns the sum of two numbers",
        "parameters": {
            "type": "object",
            "properties": {
                "a": {"type": "number", "description": "First number"},
                "b": {"type": "number", "description": "Second number"}
            },
            "required": ["a", "b"]
        }
    }
}]

# === STEP 3: Construct the initial conversation ===
# The user is asking a simple math question.
messages = [
    {"role": "system", "content": "You are a helpful assistant that uses tools to compute results."},
    {"role": "user", "content": "What's the sum of 12 and 30?"}
]

# === STEP 4: First model call — LLM decides if it needs a tool ===
response_1 = client.chat.completions.create(
    model="gpt-4-0613",
    messages=messages,
    tools=tools,
    tool_choice="auto"  # Allow the model to decide
)

mcp_msg_1 = response_1.choices[0].message

# === STEP 5: Display MCP Message 1 (Tool Call Decision) ===
print("\n================ Model Context Protocol MESSAGE 1 =================")
print(" Assistant receives the user message and evaluates:")
print("   → 'What's the sum of 12 and 30?'")

if mcp_msg_1.tool_calls:
    print(" Assistant decides to use a tool (function call)")
    tool_call = mcp_msg_1.tool_calls[0]
    print(" Tool Call ID:", tool_call.id)
    print(" Tool Name:  ", tool_call.function.name)
    print(" Arguments:  ", tool_call.function.arguments)
else:
    print(" No tool call detected")
    print("Response content:", mcp_msg_1.content)

# === STEP 6: Simulate executing the tool (e.g., backend function) ===
# The assistant requested: get_sum(a=12, b=30)
# We now simulate executing this locally in our Python environment
tool_call = mcp_msg_1.tool_calls[0]
tool_args = json.loads(tool_call.function.arguments)  # Safer than eval
result = tool_args["a"] + tool_args["b"]
tool_result = {"result": result}

print("\n Executing Tool Function:")
print(f"→ get_sum({tool_args['a']}, {tool_args['b']}) = {result}")

# === STEP 7: Add tool call + result to message history ===
# This is how we inform the assistant what the tool returned
messages.append(mcp_msg_1)  # Add the tool call message
messages.append({
    "role": "tool",
    "tool_call_id": tool_call.id,
    "content": json.dumps(tool_result)
})

# === STEP 8: Second model call — LLM integrates tool result and responds ===
response_2 = client.chat.completions.create(
    model="gpt-4-0613",
    messages=messages
)

final_response = response_2.choices[0].message.content

# === STEP 9: Display MCP Message 2 (Final Assistant Response) ===
print("\n================ Model Context Protocol MESSAGE 2 =================")
print(" Assistant receives the tool result:")
print("   → {'result': 42}")
print(" Responds to the user:")
print("   →", final_response)

# === Summary ===
print("\n================ SUMMARY =================")
print(" MCP Conversation complete:")
print("   1. Assistant received a question")
print("   2. Decided to call 'get_sum'")
print("   3. Tool returned result = 42")
print("   4. Assistant responded with the final answer")



================ Model Context Protocol MESSAGE 1 =================
 Assistant receives the user message and evaluates:
   → 'What's the sum of 12 and 30?'
 Assistant decides to use a tool (function call)
 Tool Call ID: call_JKzoHu9GnEFt9Lstv4tyAjlP
 Tool Name:   get_sum
 Arguments:   {
  "a": 12,
  "b": 30
}

 Executing Tool Function:
→ get_sum(12, 30) = 42

================ Model Context Protocol MESSAGE 2 =================
 Assistant receives the tool result:
   → {'result': 42}
 Responds to the user:
   → The sum of 12 and 30 is 42.

================ SUMMARY =================
 MCP Conversation complete:
   1. Assistant received a question
   2. Decided to call 'get_sum'
   3. Tool returned result = 42
   4. Assistant responded with the final answer


#  Message Chain Protocol Exchange Demo: Agentic AI Communication with AutoGen and Ollama

##  Goal
This section of the notebook demonstrates a **Message-Chain Protocol (MCP)**-style exchange between two agents using **AutoGen** and a **local LLM via Ollama**. The goal is to simulate how structured agent-to-agent communication unfolds in a transparent, replayable format with logs at each reasoning step.

---

##  Architecture
- **`MCPUserProxyAgent`**: Simulates a user initiating a conversation and logging acknowledgments.
- **`MCPAssistantAgent`**: Responds to the user using a locally hosted Ollama model (e.g., `llama3`) and logs its internal thought process and responses.
- **MCP Logging**: Each message turn is logged with:
  - `sender`, `receiver`
  - `thought`, `observation`
  - `timestamp` and `state_snapshot`
  - `message_id` and `turn` number

---

##  Code Overview

1. **`log_mcp_message()`**  
   Captures and prints each structured MCP message including who said what, when, and with what reasoning.

2. **LLM Configuration for Ollama**  
   Provides a locally routed API interface to a model served with Ollama (e.g., `llama3`) using OpenAI-compatible endpoints.

3. **`MCPAssistantAgent` Class**  
   - Extends `AssistantAgent` to intercept and log all replies.
   - Logs both incoming messages (thought) and generated responses (observation).

4. **`MCPUserProxyAgent` Class**  
   - Extends `UserProxyAgent` to simulate a user with one interaction round.
   - Ends the conversation after acknowledging a response from the assistant.

5. **Agent Instantiation**  
   Creates the assistant and user proxy agents with appropriate parameters:
   - Disables Docker
   - Prevents prompting in Jupyter
   - Enables default auto-reply

6. **Conversation Execution**  
   - Logs the initial user question
   - Starts the interaction
   - Captures final results or errors

7. **MCP Log Display**  
   Prints the entire exchange as structured JSON and a readable message summary.

---

##  Features Demonstrated
- **Structured Thought-Action-Observation Reasoning**
- **Replayable Message Logs**
- **Turn-Based Coordination Between Agents**
- **Integration with Local LLM (Ollama)**

---

##  Requirements
- `autogen` Python package
- Local Ollama model served via:
  ```bash
  ollama run llama3


In [27]:
# =============================================================
# Agentic AI Demo: Message Chain Protocol-Style Exchange Between Two Agents
# =============================================================

# Imports for structured messaging and time tracking
import json
from datetime import datetime
from autogen import AssistantAgent, UserProxyAgent

# === GLOBALS ===

# Stores the log of all MCP messages (thought, action, observation)
mcp_log = []

# Tracks number of messages exchanged in total
message_counter = 0

# === MCP Logging Function ===

def log_mcp_message(msg_id, sender, receiver, content, turn, is_response=False):
    """
    Records a structured Message Chain Protocol message with metadata including
    thought, observation, sender, receiver, and timestamp.
    """
    now = datetime.utcnow().isoformat()
    lines = content.strip().splitlines() if content else [""]

    # First line is treated as the agent's 'thought'
    thought = lines[0] if lines else ""
    # Remaining lines are considered the agent's 'observation'
    observation = "\n".join(lines[1:]) if len(lines) > 1 else ""

    # Construct the MCP message dictionary
    mcp_entry = {
        "message_id": f"msg_{msg_id:03}",
        "timestamp": now,
        "turn": turn,
        "sender": sender,
        "receiver": receiver,
        "thought": thought,
        "tool_call": None,         # Placeholder; could log tool name here
        "tool_response": None,     # Placeholder; could log tool output here
        "observation": observation,
        "state_snapshot": {
            "is_response": is_response,
            "conversation_stage": f"Turn {turn}"
        }
    }

    # Append the entry to the global MCP log
    mcp_log.append(mcp_entry)

    # Print the message in a readable format
    print(f"\n--- Message Chain Protocol Message {mcp_entry['message_id']} ---")
    print(f" {now}")
    print(f"From: {sender} → To: {receiver}")
    print(f"Thought: {thought}")
    if observation:
        print(f"Observation:\n{observation}")
    print("-" * 60)

# === Local LLM Configuration for Ollama ===

llm_config = {
    "config_list": [
        {
            "model": "llama3",  # Update to match the Ollama model you are running
            "base_url": "http://localhost:11434/v1",  # Ollama default endpoint
            "api_key": "ollama",                      # Dummy key for compatibility
            "price": [0.0, 0.0]                        # Avoids cost warnings
        }
    ],
    "timeout": 60,
}

# === MCP-Enabled Assistant Agent ===

class MCPAssistantAgent(AssistantAgent):
    """
    Wraps AssistantAgent to log each incoming and outgoing message as a Message Chain Protocol message.
    """
    def generate_reply(self, messages=None, sender=None, config=None):
        global message_counter
        if messages is None:
            messages = []

        # Log the incoming message (agent's 'thought')
        if messages:
            content = messages[-1].get("content", "") if isinstance(messages[-1], dict) else str(messages[-1])
            message_counter += 1
            log_mcp_message(
                msg_id=message_counter,
                sender=sender.name if sender and hasattr(sender, 'name') else "unknown",
                receiver=self.name,
                content=f"Processing request: {content}",
                turn=(message_counter + 1) // 2,
                is_response=False
            )

        # Generate actual response using parent class logic
        try:
            response = super().generate_reply(messages, sender, config)
        except TypeError:
            # Handles backwards compatibility for older AutoGen versions
            try:
                response = super().generate_reply(messages, sender)
            except:
                response = super().generate_reply(messages)

        # Extract the response content for logging
        message_counter += 1
        if isinstance(response, dict):
            response_content = response.get("content", "")
        elif isinstance(response, str):
            response_content = response
        else:
            response_content = str(response) if response else "No response generated"

        # Log the assistant's response (observation)
        log_mcp_message(
            msg_id=message_counter,
            sender=self.name,
            receiver=sender.name if sender and hasattr(sender, 'name') else "unknown",
            content=f"Generated response: {response_content}",
            turn=(message_counter + 1) // 2,
            is_response=True
        )

        # Print the assistant's response to console
        print(f"\n {self.name} Response:")
        print(f" {response_content}")
        print("-" * 40)

        return response

# === MCP-Enabled User Agent ===

class MCPUserProxyAgent(UserProxyAgent):
    """
    Wraps UserProxyAgent to automatically log responses and simulate a single turn.
    """
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.response_count = 0

    def generate_reply(self, messages=None, sender=None, config=None):
        global message_counter
        if messages is None:
            messages = []

        self.response_count += 1

        # Stop after 1 assistant response for demonstration purposes
        if self.response_count > 1:
            message_counter += 1
            log_mcp_message(
                msg_id=message_counter,
                sender=self.name,
                receiver=sender.name if sender and hasattr(sender, 'name') else "unknown",
                content="Demo exchange completed successfully",
                turn=(message_counter + 1) // 2,
                is_response=True
            )
            return None

        # Log the assistant message being processed
        if messages:
            last_message = messages[-1]
            content = last_message.get("content", "") if isinstance(last_message, dict) else str(last_message)
            message_counter += 1
            log_mcp_message(
                msg_id=message_counter,
                sender=self.name,
                receiver=sender.name if sender and hasattr(sender, 'name') else "unknown",
                content=f"Received and acknowledged: {content[:100]}...",
                turn=(message_counter + 1) // 2,
                is_response=True
            )

        # Continue the conversation (will be auto-replied)
        try:
            return super().generate_reply(messages, sender, config)
        except TypeError:
            try:
                return super().generate_reply(messages, sender)
            except:
                return super().generate_reply(messages)

# === Instantiate the Agents ===

user_proxy = MCPUserProxyAgent(
    name="user_proxy",
    human_input_mode="NEVER",                      # Prevent interactive prompts
    max_consecutive_auto_reply=1,                  # Allow one auto-response
    code_execution_config=False,                   # No Docker execution needed
    default_auto_reply="Thank you for the information!"
)

assistant = MCPAssistantAgent(
    name="climate_assistant",
    llm_config=llm_config,
    system_message="You are a helpful assistant focused on climate policy. Provide exactly two key points about the topic in a clear, concise manner."
)

# === Run the MCP Exchange ===

print("Starting Message Chain Protocol Demo Exchange...")
print("=" * 60)

try:
    # Step 1: Log the user's initial request
    initial_message = "What are two key points about global climate policy?"
    message_counter += 1
    log_mcp_message(
        msg_id=message_counter,
        sender="user_proxy",
        receiver="climate_assistant",
        content=f"Initiating conversation: {initial_message}",
        turn=1,
        is_response=False
    )

    print(f"\n User Query:")
    print(f" {initial_message}")
    print("-" * 40)

    # Step 2: Start the conversation
    result = user_proxy.initiate_chat(
        recipient=assistant,
        message=initial_message,
        max_turns=3,
        silent=False
    )

    print("\n Conversation completed successfully!")

except Exception as e:
    # If anything fails, show helpful diagnostic message
    print(f" Error during chat: {e}")
    print("This might be due to:")
    print("- Ollama not running (try: ollama serve)")
    print("- Model not available (try: ollama pull llama3)")
    print("- Network connectivity issues")
    print("- AutoGen version compatibility")

# === Show MCP Message Log ===

print("\n" + "=" * 60)
print(" Full Message Chain Protocol Exchange Log:")
print("=" * 60)
print(json.dumps(mcp_log, indent=2))

print(f"\n Total Message Chain Protocol messages logged: {len(mcp_log)}")
print(" Message Chain Protocol Exchange Summary:")
for i, entry in enumerate(mcp_log, 1):
    print(f"  {i}. {entry['sender']} → {entry['receiver']}: {entry['thought'][:50]}...")

print("\n Demo completed!")


Starting Message Chain Protocol Demo Exchange...

--- Message Chain Protocol Message msg_001 ---
 2025-06-21T15:15:40.538940
From: user_proxy → To: climate_assistant
Thought: Initiating conversation: What are two key points about global climate policy?
------------------------------------------------------------

 User Query:
 What are two key points about global climate policy?
----------------------------------------
user_proxy (to climate_assistant):

What are two key points about global climate policy?

--------------------------------------------------------------------------------

--- Message Chain Protocol Message msg_002 ---
 2025-06-21T15:15:40.540212
From: user_proxy → To: climate_assistant
Thought: Processing request: What are two key points about global climate policy?
------------------------------------------------------------

--- Message Chain Protocol Message msg_003 ---
 2025-06-21T15:15:58.872625
From: climate_assistant → To: user_proxy
Thought: Generated response: 

## Demonstrating Model Context Protocol (MCP) within a Message Chain Protocol (MCP) Workflow

### Goal
This section of the notebook demonstrates how **Model Context Protocol** (structured tool usage) can be embedded inside a **Message Chain Protocol** (explicit messaging flow) to simulate an LLM that:

- Understands a user query
- Decides to call a specific tool
- Executes that tool (e.g., adds two numbers)
- Integrates the result into its final answer
- Logs each step for traceability and interpretability

---

### Step-by-Step Breakdown

#### Step 1: Environment Setup
- Load your OpenAI API key from a local `.env` file.
- Initialize the OpenAI client.

#### Step 2: Define Tools (Model Context Protocol)
- Specify a JSON-based function (`get_sum`) that takes two numbers `a` and `b` and returns their sum.
- This definition follows the **Model Context Protocol**, which provides a schema the model can reason over.

#### Step 3: Log User Message (Message Chain Protocol)
- Record the user’s message as the first step in the **Message Chain**.
- Add metadata like role and timestamp.

#### Step 4: Let the Model Decide to Use the Tool
- Send the user question to the model.
- The model uses the **structured tool definition** to determine it needs to invoke `get_sum`.
- It responds with a `tool_call` payload (Model Context Protocol).

#### Step 5: Log Assistant’s Tool Call
- Record the model's tool call decision in the **Message Chain**.
- Include the structured arguments used in the tool call.

#### Step 6: Simulate the Tool Execution
- Use Python to simulate running the tool (`get_sum(5, 7)`).
- Capture and format the result in structured form.

#### Step 7: Log Tool Response
- Log the tool’s response into the **Message Chain Protocol** as a reply from role `tool`.

#### Step 8: Final Model Completion
- Send the original query + tool call + result to the model.
- The model replies to the user with the final answer: `The sum of 5 and 7 is 12`.

#### Step 9: Log Final Response
- Log this response in the **Message Chain Protocol**.

---

### Key Takeaway
- **Model Context Protocol** helps structure function/tool interactions.
- **Message Chain Protocol** ensures each turn (user → assistant → tool → assistant) is preserved, timestamped, and explainable.

This design is ideal for debugging, auditing, simulation, and multi-agent orchestration.



In [28]:
import os
import json
from datetime import datetime
from dotenv import load_dotenv
from openai import OpenAI

# === STEP 1: Setup ===
# Load OpenAI API key from .env (Message Chain Protocol sets environment context)
load_dotenv("keys.env")
client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

# === STEP 2: Define tool using Model Context Protocol ===
# Model Context Protocol: Structured function definition for tool use
tools = [{
    "type": "function",
    "function": {
        "name": "get_sum",
        "description": "Returns the sum of two numbers",
        "parameters": {
            "type": "object",
            "properties": {
                "a": {"type": "number", "description": "First number"},
                "b": {"type": "number", "description": "Second number"},
            },
            "required": ["a", "b"]
        }
    }
}]

# === STEP 3: Initialize message log for Message Chain Protocol ===
# Message Chain Protocol: Tracks full message history, metadata
message_log = []
message_counter = 0

def log_chain_message(role, content, model_context=None):
    """ Message Chain Protocol function to log communication steps"""
    global message_counter
    timestamp = datetime.utcnow().isoformat()
    message_counter += 1
    entry = {
        "id": f"msg_{message_counter:03}",
        "timestamp": timestamp,
        "role": role,
        "content": content,
        "model_context_protocol": model_context  # Structured model data
    }
    message_log.append(entry)

    # Display with clear distinctions
    print(f"\n================ Message Chain Message {entry['id']} =================")
    print(f" Timestamp: {timestamp}")
    print(f"Role: {role}")
    print(f"Content: {content}")
    if model_context:
        print("\n Model Context Protocol Payload:")
        print(json.dumps(model_context, indent=2))

# === STEP 4: User initiates a message ===
messages = [
    {"role": "system", "content": "You are a tool-using assistant."},
    {"role": "user", "content": "What’s the sum of 5 and 7?"}
]
log_chain_message("user", "What’s the sum of 5 and 7?")

# === STEP 5: Assistant responds with tool call (structured) ===
response_1 = client.chat.completions.create(
    model="gpt-4-0613",
    messages=messages,
    tools=tools,
    tool_choice="auto"
)
mcp_response_1 = response_1.choices[0].message

# Model Context Protocol: Tool call issued in structured format
tool_call = mcp_response_1.tool_calls[0]
tool_args = json.loads(tool_call.function.arguments)

log_chain_message("assistant", "[Tool call issued]",
    model_context={
        "tool_call": {
            "tool_name": tool_call.function.name,
            "arguments": tool_args
        }
    }
)

# === STEP 6: Simulate tool execution ===
# Model Context Protocol: Tool function executed externally
result = tool_args["a"] + tool_args["b"]
tool_result = {"result": result}

log_chain_message("tool", json.dumps(tool_result),
    model_context={
        "tool_call_id": tool_call.id,
        "tool_result": result
    }
)

# === STEP 7: Append back into messages (for assistant reply) ===
messages.append(mcp_response_1)
messages.append({
    "role": "tool",
    "tool_call_id": tool_call.id,
    "content": json.dumps(tool_result)
})

# === STEP 8: Final assistant response ===
response_2 = client.chat.completions.create(
    model="gpt-4-0613",
    messages=messages
)
final_msg = response_2.choices[0].message.content

log_chain_message("assistant", final_msg)




================ Message Chain Message msg_001 =================
 Timestamp: 2025-06-21T15:16:03.567767
Role: user
Content: What’s the sum of 5 and 7?

================ Message Chain Message msg_002 =================
 Timestamp: 2025-06-21T15:16:04.940147
Role: assistant
Content: [Tool call issued]

 Model Context Protocol Payload:
{
  "tool_call": {
    "tool_name": "get_sum",
    "arguments": {
      "a": 5,
      "b": 7
    }
  }
}

================ Message Chain Message msg_003 =================
 Timestamp: 2025-06-21T15:16:04.940643
Role: tool
Content: {"result": 12}

 Model Context Protocol Payload:
{
  "tool_call_id": "call_SQmfT4ruuVEBCsfjqDgIr2Al",
  "tool_result": 12
}

================ Message Chain Message msg_004 =================
 Timestamp: 2025-06-21T15:16:06.226804
Role: assistant
Content: The sum of 5 and 7 is 12.
